In [1]:
import pandas as pd
import os
import pickle

import networkx as nx

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from src.dataset.dataset_info import datasets
from src.graph.graph_measures import calculate_graph_measures
from src.graph.graph_construction.window_graph import create_weightless_window_graph
from src.graph.graph_construction.session_graph import define_sessions, create_weightless_session_graph

In [2]:
# name = "cic_ton_iot_5_percent"
# name = "cic_ton_iot"
# name = "cic_ids_2017_5_percent"
# name = "cic_ids_2017"
name = "cic_bot_iot"
# name = "cic_ton_iot_modified"
# name = "nf_ton_iotv2_modified"
# name = "ccd_inid_modified"
# name = "nf_uq_nids_modified"
# name = "edge_iiot"
# name = "nf_cse_cic_ids2018"
# name = "nf_bot_iotv2"
# name = "nf_uq_nids"
# name = "x_iiot"

dataset = datasets[name]

In [3]:
with open("datasets/" + name + "/training_graph.pkl", "rb") as f:
    G = pickle.load(f)

In [ ]:
import igraph as ig
G1 = ig.Graph.from_networkx(G)

part = G1.community_infomap()

In [4]:
import networkx as nx

import timeit

verbose = True
properties = {}

start_time = timeit.default_timer()

# part = G1.community_multilevel()
# part = G1.community_spinglass()
# part = G1.community_edge_betweenness()

communities = []
for com in part:
    communities.append([G1.vs[node_index]['_nx_name']
                        for node_index in com])

# communities = nx.community.louvain_communities(G)
number_of_communities = len(communities)
if verbose:
    print(
        f"==>> number_of_communities: {number_of_communities}, in {str(timeit.default_timer() - start_time)} seconds")
properties["number_of_communities"] = number_of_communities

# Step 1: Map each node to its community
node_to_community = {}
for community_index, community in enumerate(communities):
    for node in community:
        node_to_community[node] = community_index

# Step 2: Count inter-cluster edges efficiently
inter_cluster_edges = 0
for u, v in G.edges():
    # Directly check if u and v belong to different communities
    if node_to_community[u] != node_to_community[v]:
        inter_cluster_edges += 1

start_time = timeit.default_timer()
mixing_parameter = inter_cluster_edges / G.number_of_edges()
if verbose:
    print(
        f"==>> mixing_parameter: {mixing_parameter}, in {str(timeit.default_timer() - start_time)} seconds")
properties["mixing_parameter"] = mixing_parameter

start_time = timeit.default_timer()
modularity = nx.community.modularity(G, communities)
if verbose:
    print(
        f"==>> modularity: {modularity}, in {str(timeit.default_timer() - start_time)} seconds")
properties["modularity"] = modularity